In [1]:
!neofetch

            .-/+oossssoo+/-.
        `:+ssssssssssssssssss+:`
      -+ssssssssssssssssssyyssss+-
    .ossssssssssssssssssdMMMNysssso.
   /ssssssssssshdmmNNmmyNMMMMhssssss/
  +ssssssssshmydMMMMMMMNddddyssssssss+
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/
.ssssssssdMMMNhsssssssssshNMMMdssssssss.
+sssshhhyNMMNyssssssssssssyNMMMysssssss+
ossyNMMMNyMMhsssssssssssssshmmmhssssssso
ossyNMMMNyMMhsssssssssssssshmmmhssssssso
+sssshhhyNMMNyssssssssssssyNMMMysssssss+
.ssssssssdMMMNhsssssssssshNMMMdssssssss.
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/
  +sssssssssdmydMMMMMMMMddddyssssssss+
   /ssssssssssshdmNNNNmyNMMMMhssssss/
    .ossssssssssssssssssdMMMNysssso.
      -+sssssssssssssssssyyyssss+-
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.
ragu@MATRIX 
----------- 
OS: Ubuntu 24.04.1 LTS x86_64 
Host: ROG Zephyrus G14 GA401IH_GA401IH 1.0 
Kernel: 6.8.0-48-generic 
Uptime: 2 hours, 2 mins 
Packages: 2629 (dpkg), 15 (snap) 
Shell: zsh 5.9 
Resolution: 1920x1080 
DE: GNOME 46.0 
WM: Mutter 

In [2]:
! cat Scripts/requirements.txt

numpy
pandas
matplotlib
tensorflow
scikit-learn
Pillow
seaborn

In [3]:
import os
import shutil
import glob
import logging
import subprocess
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from Scripts.ImageAug import process_dataset  # Assuming process_dataset function is defined in ImageAug
from Scripts.ExtractFeatures import extract_features  # Assuming extract_features function is defined in ExtractFeatures
from Scripts.Models import evaluate_models  # Assuming evaluate_models function is defined in Models

2024-11-08 12:47:46.876984: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-08 12:47:46.882363: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-08 12:47:46.899481: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-08 12:47:46.926831: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-08 12:47:46.934902: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 12:47:46.955955: I tensorflow/core/platform/cpu_feature_gu

In [4]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [5]:
def list_directory_contents(directory):
    """List the contents of a directory."""
    logging.info(f"Contents of {directory}:")
    contents = os.listdir(directory)
    for item in contents:
        logging.info(item)

In [6]:
def create_and_process_dataset(dataset_type, num_augmentations=1):
    """Create and process the dataset."""
    os.system(f"./Scripts/create_dataset.sh {dataset_type}")
    logging.info("Dataset creation script executed for %s", dataset_type)
    
    dataset_dir = f"{dataset_type.capitalize()}_Dataset/"
    
    # Check if the dataset directory was created
    if not os.path.exists(dataset_dir):
        logging.error("Dataset directory not found: %s", dataset_dir)
        raise FileNotFoundError(f"Dataset directory '{dataset_dir}' does not exist.")
    
    list_directory_contents(dataset_dir)
    
    process_dataset(dataset_dir, num_augmentations=num_augmentations)
    logging.info("Processing completed for %s", dataset_dir)
    
    return 'Augmented_DataSet/'

In [7]:
def extract_features_for_models(data_dir):
    """Extract features using various models."""
    models = ['ResNet50', 'InceptionV3', 'MobileNetV2', 'DenseNet121', 'EfficientNetB0']
    for model in models:
        extract_features(model, data_dir)
        logging.info("Extracted features using %s", model)

In [8]:
def evaluate_models_and_save_results(model_params, output_filename, class_names, dataset_type):
    """Evaluate models, save results, and save models to directory."""
    results = evaluate_models(model_params=model_params, class_names=class_names)
    
    # Ensure Reports directory exists before renaming
    if not os.path.exists("Reports"):
        os.makedirs("Reports")
    
    # Save the results to the specified output filename
    os.rename("Reports/model_results.csv", output_filename)
    logging.info("Model results saved to %s", output_filename)

    # Move saved models to specific directory for this dataset type
    move_models(dataset_type)

In [9]:
def move_cm(dir_name):
    """Move confusion matrices to a directory named after the dataset type."""
    os.makedirs(f"Conf_Matrix/{dir_name}", exist_ok=True)
    for file_path in glob.glob("Conf_Matrix/*.png"):
        shutil.move(file_path, f"Conf_Matrix/{dir_name}/")
        print(f'Moved: {file_path} to Conf_Matrix/{dir_name}/')

In [10]:
def move_models(dir_name):
    """Move saved models to a directory named after the dataset type."""
    os.makedirs(f"Models/{dir_name}", exist_ok=True)
    for file_path in glob.glob("Models/*.pkl"):  # Assuming models are saved as .pkl files
        shutil.move(file_path, f"Models/{dir_name}/")
        print(f'Moved: {file_path} to Models/{dir_name}/')

In [11]:
def main():
    # List contents of Original_DataSet
    list_directory_contents("Original_DataSet/")

    # Define model parameters
    model_params = {
        SVC: [{'kernel': 'rbf'}, {'kernel': 'linear'}],
        DecisionTreeClassifier: [{'max_depth': None}, {'max_depth': 5}],
        RandomForestClassifier: [{'n_estimators': 100, 'max_depth': None, 'random_state': 42}],
        KNeighborsClassifier: [{'n_neighbors': 5}]
    }

    # Dataset configurations: dataset type, result filename, and class names for each type
    datasets = [
        {'type': 'binary', 'filename': 'Reports/model_results_binary.csv', 'class_names': ['Healthy', 'Reject']},
        {'type': 'multiclass1', 'filename': 'Reports/model_results_three_class.csv', 'class_names': ['Ripe', 'Unripe', 'Reject']},
        {'type': 'multiclass2', 'filename': 'Reports/model_results_four_class.csv', 'class_names': ['Ripe', 'Unripe', 'Old', 'Damaged']}
    ]

    # Create, process datasets, extract features, evaluate models, and organize results
    for dataset in datasets:
        data_dir = create_and_process_dataset(dataset['type'])
        extract_features_for_models(data_dir)
        evaluate_models_and_save_results(
            model_params=model_params,
            output_filename=dataset['filename'],
            class_names=dataset['class_names'],
            dataset_type=dataset['type']
        )
        move_cm(dataset['type'])

    logging.info("#################### Completed ##########################")
    # Command to remove the specified directories
    command = "rm -rf Augmented_DataSet Binary_Dataset Features Multiclass1_Dataset Multiclass2_Dataset"

    # Execute the command in a shell
    try:
        subprocess.run(command, shell=True, check=True)
        print("Directories removed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred: {e}")

In [12]:
if __name__ == "__main__":
    main()

2024-11-08 12:47:49,431 - INFO - Contents of Original_DataSet/:
2024-11-08 12:47:49,433 - INFO - Ripe
2024-11-08 12:47:49,434 - INFO - Damaged
2024-11-08 12:47:49,436 - INFO - Unripe
2024-11-08 12:47:49,436 - INFO - Old
2024-11-08 12:47:49,495 - INFO - Dataset creation script executed for binary
2024-11-08 12:47:49,496 - INFO - Contents of Binary_Dataset/:
2024-11-08 12:47:49,497 - INFO - Reject
2024-11-08 12:47:49,497 - INFO - Healthy


Binary Dataset creation Completed
Saved Augmented_DataSet/Reject/augmented_1_o (2).png
Saved Augmented_DataSet/Reject/augmented_1_D015.png
Saved Augmented_DataSet/Reject/augmented_1_o (6).png
Saved Augmented_DataSet/Reject/augmented_1_o (3).png
Saved Augmented_DataSet/Reject/augmented_1_D004.png
Saved Augmented_DataSet/Reject/augmented_1_o (7).png
Saved Augmented_DataSet/Reject/augmented_1_D003.png
Saved Augmented_DataSet/Reject/augmented_1_o (1).png
Saved Augmented_DataSet/Reject/augmented_1_D011.png
Saved Augmented_DataSet/Reject/augmented_1_o (5).png
Saved Augmented_DataSet/Reject/augmented_1_o (8).png
Saved Augmented_DataSet/Reject/augmented_1_D006.png
Saved Augmented_DataSet/Reject/augmented_1_D002.png
Saved Augmented_DataSet/Reject/augmented_1_D009.png
Saved Augmented_DataSet/Reject/augmented_1_D007.png
Saved Augmented_DataSet/Reject/augmented_1_D012.png
Saved Augmented_DataSet/Reject/augmented_1_o (4).png
Saved Augmented_DataSet/Healthy/augmented_1_r (434).png
Saved Augmented_Da

2024-11-08 12:47:51,063 - INFO - Processing completed for Binary_Dataset/


Saved Augmented_DataSet/Healthy/augmented_1_UNRIPE-015.png


I0000 00:00:1731050271.101895   17905 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-08 12:47:51.103060: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


...

2024-11-08 12:48:17,317 - INFO - Extracted features using EfficientNetB0


Features and labels extracted using EfficientNetB0 and saved successfully in 'Features' directory.

 Loaded ResNet50 Features ! 

Model: SVC with parameters [kernel: rbf] saved as Models/ResNet50_SVC_kernel: rbf.pkl
Model: SVC with parameters [kernel: linear] saved as Models/ResNet50_SVC_kernel: linear.pkl
Model: Decision Tree with parameters [max_depth: None] saved as Models/ResNet50_Decision Tree_max_depth: None.pkl
Model: Decision Tree with parameters [max_depth: 5] saved as Models/ResNet50_Decision Tree_max_depth: 5.pkl
Model: Random Forest with parameters [n_estimators: 100, max_depth: None, random_state: 42] saved as Models/ResNet50_Random Forest_n_estimators: 100, max_depth: None, random_state: 42.pkl
Model: KNN with parameters [n_neighbors: 5] saved as Models/ResNet50_KNN_n_neighbors: 5.pkl

 Loaded InceptionV3 Features ! 

Model: SVC with parameters [kernel: rbf] saved as Models/InceptionV3_SVC_kernel: rbf.pkl
Model: SVC with parameters [kernel: linear] saved as Models/Incepti

2024-11-08 12:48:26,177 - INFO - Model results saved to Reports/model_results_binary.csv


Model: KNN with parameters [n_neighbors: 5] saved as Models/EfficientNetB0_KNN_n_neighbors: 5.pkl


Error: Destination path 'Models/binary/InceptionV3_Decision Tree_max_depth: 5.pkl' already exists

In [ ]:
!rm 